<a href="https://colab.research.google.com/github/omarcesarcq/otro-ejemplo-git/blob/main/cliente_mqtt_microPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import machine
import time
import network
from umqtt.simple import MQTTClient

# Configuración de la red Wi-Fi
wifi_ssid = "HUAWEI_MATE8"
wifi_password = "d395db00"

# Configuración del servidor MQTT
mqtt_server = "192.168.43.135"
mqtt_topic = "/prueba"

led_pin = machine.Pin(15, machine.Pin.OUT)  # Pin GPIO2 para el LED

def connect_to_wifi():
    sta_if = network.WLAN(network.STA_IF)
    sta_if.active(True)
    if not sta_if.isconnected():
        print("Conectando a la red Wi-Fi...")
        sta_if.connect(wifi_ssid, wifi_password)
        while not sta_if.isconnected():
            print("*")
            time.sleep(1)
    print("Conectado a la red Wi-Fi:", sta_if.ifconfig()[0])

def on_message(topic, message):
    if topic.decode() == mqtt_topic.decode():
        if message.decode() == "ON":
            led_pin.on()
        elif message.decode() == "OFF":
            led_pin.off()

def connect_to_mqtt():
    client_id = "esp8266_1"
    mqtt_user = "usuario1"
    mqtt_pass = "usuario1"
    client = MQTTClient(client_id, mqtt_server, user=mqtt_user, password=mqtt_pass)
    client.set_callback(on_message)
    client.connect()
    client.subscribe(mqtt_topic)
    print("Conectado al servidor MQTT")

    return client

def main():
    connect_to_wifi()
    client = connect_to_mqtt()

    while True:
        try:
            client.check_msg()


        except OSError:
            print("Error de conexión MQTT. Reconectando...")
            client = connect_to_mqtt()

if __name__ == "__main__":
    main()